In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
import re
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, GRU, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from keras.metrics import Precision, Recall
from keras.callbacks import Callback
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Load the csv files
df1 = pd.read_csv("dailydialog.csv")
df2 = pd.read_csv("emotion-stimulus.csv")
df3 = pd.read_csv("isear.csv")
df4 = pd.read_csv("emotion_dataset.csv")
df5 = pd.read_csv("emotion_dataset2.csv")
df6 = pd.read_csv("emotion-tweets.csv")

In [3]:
# Concatenate DataFrames row-wise
df = pd.concat([df1, df2, df3,df4,df5,df6], axis=0)

In [4]:
#Display the count of all values
df['Emotion'].value_counts()

Emotion
neutral      87826
joy          20503
sadness      15119
Happiness    12885
fear         11539
anger        10261
surprise      4994
disgust       2018
Surpise       1823
shame         1346
love          1304
Sadness       1150
guilt         1052
Anger         1022
sad            575
happy          479
Disgust        353
love           337
Fear           174
Name: count, dtype: int64

In [5]:
"""# Check for duplicate rows
duplicates = df['Emotion'].duplicated()

# Print the rows that are duplicates
print(df[duplicates])

# Count the number of duplicate rows
num_duplicates = duplicates.sum()
print(f"Number of duplicate rows: {num_duplicates}")"""


'# Check for duplicate rows\nduplicates = df[\'Emotion\'].duplicated()\n\n# Print the rows that are duplicates\nprint(df[duplicates])\n\n# Count the number of duplicate rows\nnum_duplicates = duplicates.sum()\nprint(f"Number of duplicate rows: {num_duplicates}")'

In [6]:
"""# Remove duplicate rows
df = df.drop_duplicates()

# Reset the index after removing duplicates (optional)
df = df.reset_index(drop=True)"""

'# Remove duplicate rows\ndf = df.drop_duplicates()\n\n# Reset the index after removing duplicates (optional)\ndf = df.reset_index(drop=True)'

In [7]:
"""# Check for duplicate rows
duplicates = df.duplicated()

# Print the rows that are duplicates
print(df[duplicates])

# Count the number of duplicate rows
num_duplicates = duplicates.sum()
print(f"Number of duplicate rows: {num_duplicates}")"""


'# Check for duplicate rows\nduplicates = df.duplicated()\n\n# Print the rows that are duplicates\nprint(df[duplicates])\n\n# Count the number of duplicate rows\nnum_duplicates = duplicates.sum()\nprint(f"Number of duplicate rows: {num_duplicates}")'

In [8]:
#Display the count of all values
df['Emotion'].value_counts()

Emotion
neutral      87826
joy          20503
sadness      15119
Happiness    12885
fear         11539
anger        10261
surprise      4994
disgust       2018
Surpise       1823
shame         1346
love          1304
Sadness       1150
guilt         1052
Anger         1022
sad            575
happy          479
Disgust        353
love           337
Fear           174
Name: count, dtype: int64

In [9]:
# Keeping only 6 labels to simplify
df = df[~df['Emotion'].str.contains('love', case=False, na=False)]
df = df[~df['Emotion'].str.contains('shame', case=False, na=False)]
df = df[~df['Emotion'].str.contains('disgust', case=False, na=False)]
df = df[~df['Emotion'].str.contains('guilt', case=False, na=False)]
df = df[~df['Emotion'].str.contains('Surpise', case=False, na=False)]
df = df[~df['Emotion'].str.contains('surprise', case=False, na=False)]

In [10]:
#Display the count of all values
df['Emotion'].value_counts()

Emotion
neutral      87826
joy          20503
sadness      15119
Happiness    12885
fear         11539
anger        10261
Sadness       1150
Anger         1022
sad            575
happy          479
Fear           174
Name: count, dtype: int64

In [11]:
# Count the NaN values in each column
nan_counts = df.isna().sum()
print("Number of NaN values per column:")
print(nan_counts)

# Count the null values in each column (similar to isna(), but shown for illustration)
null_counts = df.isnull().sum()
print("\nNumber of null values per column:")
print(null_counts)

# Display the total number of NaN values in the dataset
total_nan_counts = df.isna().sum().sum()
print(f'\nTotal number of NaN values in the dataset: {total_nan_counts}')

# Display the total number of null values in the dataset
total_null_counts = df.isnull().sum().sum()
print(f'Total number of null values in the dataset: {total_null_counts}')


Number of NaN values per column:
Emotion    0
Text       0
dtype: int64

Number of null values per column:
Emotion    0
Text       0
dtype: int64

Total number of NaN values in the dataset: 0
Total number of null values in the dataset: 0


In [12]:
"""# Remove duplicate rows
df = df.drop_duplicates()

# Reset the index after removing duplicates (optional)
df = df.reset_index(drop=True)"""

'# Remove duplicate rows\ndf = df.drop_duplicates()\n\n# Reset the index after removing duplicates (optional)\ndf = df.reset_index(drop=True)'

In [13]:
"""# Check for duplicate rows
duplicates = df.duplicated()

# Print the rows that are duplicates
print(df[duplicates])

# Count the number of duplicate rows
num_duplicates = duplicates.sum()
print(f"Number of duplicate rows: {num_duplicates}")"""


'# Check for duplicate rows\nduplicates = df.duplicated()\n\n# Print the rows that are duplicates\nprint(df[duplicates])\n\n# Count the number of duplicate rows\nnum_duplicates = duplicates.sum()\nprint(f"Number of duplicate rows: {num_duplicates}")'

In [14]:
print("Unique values in 'Emotion' column before mapping:")
print(df['Emotion'].unique())

Unique values in 'Emotion' column before mapping:
['neutral' 'Happiness' 'Fear' 'Sadness' 'Anger' 'happy' 'sad' 'anger'
 'fear' 'joy' 'sadness']


In [15]:
import pandas as pd

# Define a mapping to unify redundant labels
label_mapping = {
    'Happiness': 'Happiness',
    'happy': 'Happiness',
    'Surprise': 'surprise',
    'surprise': 'surprise',
    'joy' :'joy',
    'Sadness': 'sadness',
    'sad': 'sadness',
    'sadness': 'sadness',
    'Fear': 'fear',
    'fear': 'fear',
    'Anger': 'anger',
    'anger': 'anger',
    'Disgust': 'disgust',
    'disgust': 'disgust',
    'shame': 'shame',
    'guilt': 'guilt',
    'neutral': 'neutral'
}

# Apply the mapping to the 'Emotion' column
df['Emotion'] = df['Emotion'].map(label_mapping)

# Check for unmapped labels (if any)
unmapped = df[df['Emotion'].isnull()]['Emotion'].unique()
if len(unmapped) > 0:
    print(f"Unmapped labels: {unmapped}")
else:
    print("All labels were successfully normalized.")

# Check the unique labels after normalization
print("Unique labels after normalization:")
print(df['Emotion'].unique())


All labels were successfully normalized.
Unique labels after normalization:
['neutral' 'Happiness' 'fear' 'sadness' 'anger' 'joy']


In [16]:
#Display the count of all values of emotion data
df['Emotion'].value_counts()

Emotion
neutral      87826
joy          20503
sadness      16844
Happiness    13364
fear         11713
anger        11283
Name: count, dtype: int64

In [17]:
# Count the NaN values in each column
nan_counts = df.isna().sum()
print("Number of NaN values per column:")
print(nan_counts)

# Count the null values in each column (similar to isna(), but shown for illustration)
null_counts = df.isnull().sum()
print("\nNumber of null values per column:")
print(null_counts)

# Display the total number of NaN values in the dataset
total_nan_counts = df.isna().sum().sum()
print(f'\nTotal number of NaN values in the dataset: {total_nan_counts}')

# Display the total number of null values in the dataset
total_null_counts = df.isnull().sum().sum()
print(f'Total number of null values in the dataset: {total_null_counts}')


Number of NaN values per column:
Emotion    0
Text       0
dtype: int64

Number of null values per column:
Emotion    0
Text       0
dtype: int64

Total number of NaN values in the dataset: 0
Total number of null values in the dataset: 0


In [18]:
"""# Drop rows where 'Emotion' has NaN values
df = df.dropna(subset=['Emotion'])"""

"# Drop rows where 'Emotion' has NaN values\ndf = df.dropna(subset=['Emotion'])"

In [19]:
#Display the count of all values of emotion data
df['Emotion'].value_counts()


Emotion
neutral      87826
joy          20503
sadness      16844
Happiness    13364
fear         11713
anger        11283
Name: count, dtype: int64

In [20]:
df.head()

,Emotion,Text
0,neutral,"Say , Jim , how about going for a few beers af..."
1,neutral,You know that is tempting but is really not g...
2,neutral,What do you mean ? It will help us to relax .
3,neutral,Do you really think so ? I don't . It will ju...
4,neutral,I guess you are right.But what shall we do ? ...


In [21]:
# Count the NaN values in each column
nan_counts = df.isna().sum()
print("Number of NaN values per column:")
print(nan_counts)

# Count the null values in each column (similar to isna(), but shown for illustration)
null_counts = df.isnull().sum()
print("\nNumber of null values per column:")
print(null_counts)

# Display the total number of NaN values in the dataset
total_nan_counts = df.isna().sum().sum()
print(f'\nTotal number of NaN values in the dataset: {total_nan_counts}')

# Display the total number of null values in the dataset
total_null_counts = df.isnull().sum().sum()
print(f'Total number of null values in the dataset: {total_null_counts}')


Number of NaN values per column:
Emotion    0
Text       0
dtype: int64

Number of null values per column:
Emotion    0
Text       0
dtype: int64

Total number of NaN values in the dataset: 0
Total number of null values in the dataset: 0


In [22]:
import pandas as pd

# Function to sample a specific number of entries for each emotion
def sample_emotions(df, emotions_to_sample, desired_count):
    sampled_dfs = []
    for emotion in emotions_to_sample:
        # Filter entries for the specified emotion
        emotion_comments = df[df['Emotion'] == emotion]
        current_count = len(emotion_comments)
        print(f"Current number of '{emotion}' comments: {current_count}")

        # Check if there are enough entries to sample
        if desired_count > current_count:
            raise ValueError(f"Desired count for '{emotion}' exceeds available entries.")

        # Randomly sample the desired number of entries
        sampled_dfs.append(emotion_comments.sample(n=desired_count, random_state=42))
    return pd.concat(sampled_dfs)

# List of emotions to sample and desired count
emotions_to_sample = ['neutral','Happiness','joy','sadness','fear','anger']
desired_count = 11000

# Sample the specified number of entries for each emotion
sampled_emotions = sample_emotions(df, emotions_to_sample, desired_count)

# Select all entries that are not in the sampled emotions
other_comments = df[~df['Emotion'].isin(emotions_to_sample)]

# Combine the sampled entries with the other comments
df = pd.concat([sampled_emotions, other_comments])

# Shuffle the combined DataFrame
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Verify the new class distribution
print(df['Emotion'].value_counts())


Current number of 'neutral' comments: 87826
Current number of 'Happiness' comments: 13364
Current number of 'joy' comments: 20503
Current number of 'sadness' comments: 16844
Current number of 'fear' comments: 11713
Current number of 'anger' comments: 11283
Emotion
Happiness    11000
sadness      11000
anger        11000
joy          11000
fear         11000
neutral      11000
Name: count, dtype: int64


In [23]:
# Display class distribution
class_counts = df['Emotion'].value_counts()
print("Class distribution before augmentation:\n", class_counts)

Class distribution before augmentation:
 Emotion
Happiness    11000
sadness      11000
anger        11000
joy          11000
fear         11000
neutral      11000
Name: count, dtype: int64


In [24]:
import pandas as pd
import nlpaug.augmenter.word as naw

# Initialize the Synonym-based augmenter
augmenter = naw.SynonymAug(aug_src='wordnet')

# Function to augment text using SynonymAug
def augment_text_synonym(text, augmenter, n=1):
    augmented_texts = [augmenter.augment(text) for _ in range(n)]
    return augmented_texts

# Set the minimum target count per class
minimum_target_count = 17000

# List to hold augmented data
augmented_data = []

# Augment data for each class
for emotion, count in class_counts.items():
    # Determine the number of samples to augment
    num_augmentations = max(0, minimum_target_count - count)  # Ensure it's non-negative

    if num_augmentations > 0:
        # Filter the DataFrame for the current emotion
        df_emotion = df[df['Emotion'] == emotion]

        while num_augmentations > 0:
            for _, row in df_emotion.iterrows():
                # Stop if we've reached the target
                if num_augmentations <= 0:
                    break

                # Generate augmented texts
                augmented_texts = augment_text_synonym(row['Text'], augmenter, n=1)  # Generate one at a time

                for aug_text in augmented_texts:
                    # Ensure `aug_text` is a string
                    if isinstance(aug_text, list):
                        aug_text = aug_text[0]  # Extract the string from the list
                    augmented_data.append({'Text': aug_text, 'Emotion': emotion})
                    num_augmentations -= 1  # Reduce the count needed
                    if num_augmentations <= 0:
                        break

# Create a DataFrame for the augmented data
augmented_df = pd.DataFrame(augmented_data)

# Combine the original DataFrame with the augmented data
df = pd.concat([df, augmented_df], ignore_index=True)

# Display class distribution after augmentation
class_counts_after_augmentation = df['Emotion'].value_counts()
print("Class distribution after augmentation:\n", class_counts_after_augmentation)


Class distribution after augmentation:
 Emotion
Happiness    17000
sadness      17000
anger        17000
joy          17000
fear         17000
neutral      17000
Name: count, dtype: int64


In [25]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [26]:
"""# Check for duplicate rows
duplicates = df.duplicated()

# Print the rows that are duplicates
print("Duplicate rows:")
print(df[duplicates])

# Count the number of duplicate rows
num_duplicates = duplicates.sum()
print(f"Number of duplicate rows: {num_duplicates}")"""


'# Check for duplicate rows\nduplicates = df.duplicated()\n\n# Print the rows that are duplicates\nprint("Duplicate rows:")\nprint(df[duplicates])\n\n# Count the number of duplicate rows\nnum_duplicates = duplicates.sum()\nprint(f"Number of duplicate rows: {num_duplicates}")'

In [27]:
"""# Remove duplicate rows
df = df.drop_duplicates()

# Reset the index after removing duplicates (optional)
df = df.reset_index(drop=True)"""


'# Remove duplicate rows\ndf = df.drop_duplicates()\n\n# Reset the index after removing duplicates (optional)\ndf = df.reset_index(drop=True)'

In [28]:
"""# Display class distribution
class_counts = df['Emotion'].value_counts()
print("Class distribution before augmentation:\n", class_counts)"""

'# Display class distribution\nclass_counts = df[\'Emotion\'].value_counts()\nprint("Class distribution before augmentation:\n", class_counts)'

In [29]:
"""# Check for duplicate rows
duplicates = df.duplicated()

# Print the rows that are duplicates
print("Duplicate rows:")
print(df[duplicates])

# Count the number of duplicate rows
num_duplicates = duplicates.sum()
print(f"Number of duplicate rows: {num_duplicates}")"""


'# Check for duplicate rows\nduplicates = df.duplicated()\n\n# Print the rows that are duplicates\nprint("Duplicate rows:")\nprint(df[duplicates])\n\n# Count the number of duplicate rows\nnum_duplicates = duplicates.sum()\nprint(f"Number of duplicate rows: {num_duplicates}")'

In [30]:
#Save the augmented dataset
df.to_csv('augmented_dataset_dialog_samy.csv', index=False)

In [31]:
import pandas as pd

# Mapping emotions to sentiments
sentiment_map = {
    'neutral': 'neutral',
    'Happiness': 'positive',
    'surprise': 'positive',
    'joy' : 'positive',
    'sadness': 'negative',
    'anger': 'negative',
    'fear': 'negative',
    'disgust': 'negative',
    'shame': 'negative',
    'guilt': 'negative'
    
}

# Create a new column 'Sentiment' by mapping the 'Emotion' column
df['Sentiment'] = df['Emotion'].map(sentiment_map)

# Check for unmapped emotions (optional debugging step)
unmapped = df[df['Sentiment'].isnull()]['Emotion'].unique()
if len(unmapped) > 0:
    print(f"Unmapped emotions: {unmapped}")
else:
    print("All emotions were successfully mapped to sentiments.")

# Display the updated DataFrame
print(df.head())


All emotions were successfully mapped to sentiments.
     Emotion                                               Text Sentiment
0  Happiness                                      Yes . When ?   positive
1    sadness  I don't think I could cope with your heartbrea...  negative
2    sadness                 Lundi y a J Cole je n'y serai pas   negative
3  Happiness   It ’ s just the cows that are grazing over th...  positive
4      anger  @MagicAndFangs 'Just by getting lost! I don't ...  negative


In [32]:
# Define a mapping for emotions to unique integer codes
emotion_mapping = {
    'fear': 0,
    'Happiness': 1,
    'anger': 2,
    'joy' : 3,   
    'sadness': 4,    
    'neutral': 5, 
}

# Map the 'Emotion' column to integer codes
df['Emotion'] = df['Emotion'].map(emotion_mapping)

# Check for unmapped values in Emotion
unmapped_emotions = df[df['Emotion'].isnull()]['Emotion'].unique()
if len(unmapped_emotions) > 0:
    print(f"Unmapped emotions: {unmapped_emotions}")
else:
    print("All emotions were successfully mapped.")


All emotions were successfully mapped.


In [33]:
# Define a mapping for sentiments to unique integer codes
sentiment_mapping = {
    'neutral': 0,
    'positive': 1,
    'negative': 2
}

# Map the 'Sentiment' column to integer codes
df['Sentiment'] = df['Sentiment'].map(sentiment_mapping)

# Check for unmapped values in Sentiment
unmapped_sentiments = df[df['Sentiment'].isnull()]['Sentiment'].unique()
if len(unmapped_sentiments) > 0:
    print(f"Unmapped sentiments: {unmapped_sentiments}")
else:
    print("All sentiments were successfully mapped.")


All sentiments were successfully mapped.


In [34]:
# Count the NaN values in each column
nan_counts = df.isna().sum()
print("Number of NaN values per column:")
print(nan_counts)

# Count the null values in each column (similar to isna(), but shown for illustration)
null_counts = df.isnull().sum()
print("\nNumber of null values per column:")
print(null_counts)

# Display the total number of NaN values in the dataset
total_nan_counts = df.isna().sum().sum()
print(f'\nTotal number of NaN values in the dataset: {total_nan_counts}')

# Display the total number of null values in the dataset
total_null_counts = df.isnull().sum().sum()
print(f'Total number of null values in the dataset: {total_null_counts}')


Number of NaN values per column:
Emotion      0
Text         0
Sentiment    0
dtype: int64

Number of null values per column:
Emotion      0
Text         0
Sentiment    0
dtype: int64

Total number of NaN values in the dataset: 0
Total number of null values in the dataset: 0


In [35]:
# Reorder the columns
df = df[['Text', 'Emotion', 'Sentiment']]
# Display the updated DataFrame
df.head()


,Text,Emotion,Sentiment
0,Yes . When ?,1,1
1,I don't think I could cope with your heartbrea...,4,2
2,Lundi y a J Cole je n'y serai pas,4,2
3,It ’ s just the cows that are grazing over th...,1,1
4,@MagicAndFangs 'Just by getting lost! I don't ...,2,2


In [3]:
import re
from nltk.tokenize import word_tokenize

def clean_text(text):
    # Check if text is a string (to avoid errors with NaNs)
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        
        # Remove URLs
        text = re.sub(r"http\S+|www\S+", '', text)
        
        # Remove hashtags and @usernames
        text = re.sub(r"(#[\d\w\.]+)", '', text)
        text = re.sub(r"(@[\d\w\.]+)", '', text)
        
        # Remove numbers (optional if numbers are relevant)
        text = re.sub(r"\d+", '', text)
        
        # **Remove underscores**
        text = text.replace('_', '')
        
        # Remove all special characters except valid punctuation (if needed)
        text = re.sub(r"[^\w\s]", '', text)
        
        # Remove non-ASCII characters
        text = re.sub(r"[^\x00-\x7F]+", '', text)
        
        # Remove extra whitespaces
        text = re.sub(r"\s+", ' ', text).strip()

        # Tokenization using nltk
        tokens = word_tokenize(text)

        # Remove stop words (if you have a defined stop words list)
        # Uncomment and define stop words if needed:
        # stop_words = set(stopwords.words('english'))
        # tokens = [word for word in tokens if word not in stop_words]

        # Return the cleaned and tokenized text
        return tokens
    
    return text  # Return the input as is if it's not a string


In [37]:
# Apply the cleaning function to the 'Text' column
df['Text'] = df['Text'].apply(clean_text)

In [38]:
#Save the bclean augmented dataset
df.to_csv('cleaned_augmented_dataset_dialog_samy.csv', index=False)

In [39]:
df.head(10)

,Text,Emotion,Sentiment
0,"[yes, when]",1,1
1,"[i, dont, think, i, could, cope, with, your, h...",4,2
2,"[lundi, y, a, j, cole, je, ny, serai, pas]",4,2
3,"[it, s, just, the, cows, that, are, grazing, o...",1,1
4,"[just, by, getting, lost, i, dont, want, to, s...",2,2
5,"[unbeknown, to, me, my, flatmate, has, become,...",3,1
6,"[i, feel, will, be, warmly, welcomed, on, any,...",3,1
7,"[im, going, to, do, some, shopping, why, dont,...",1,1
8,"[i, have, lemonade, iced, tea, and, mango, juice]",1,1
9,"[when, a, republican, says, shining, city, i, ...",0,2


In [40]:
print(df['Text'][120])

['im', 'sorry', 'i', 'didnt', 'quite', 'catch', 'that']


In [41]:
import pandas as pd
import numpy as np

# Extract 'Emotion' and 'Sentiment' as labels
y = df[['Emotion', 'Sentiment']]  

# Extract 'Text' as features
X = df['Text']  


In [42]:
# Split 80% for training and 20% for temporary set (which will be split into validation and test sets)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the 20% temporary set into 10% validation and 10% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [43]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import layers, models, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tqdm import tqdm
from transformers import BertTokenizer,RobertaTokenizer
# Initialize the tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the texts
tokenizer.fit_on_texts(X)

# Extract word index and vocabulary size
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 45035


In [44]:
#Convert text into numerical values
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(X_test)
#Calculate the max lengh of text
max_length = 0
for sequence in X_train:
    sequence_length = len(sequence)
    if sequence_length > max_length:
        max_length = sequence_length


print(max_length)

236


In [45]:
#Adding zeros at the beginning
X_train = pad_sequences(X_train,maxlen=max_length)
X_val = pad_sequences(X_val,maxlen=max_length)
X_test = pad_sequences(X_test,maxlen=max_length)

In [46]:
#Calculate the max vocabulary size
vocab_size = len(word_index) + 1
vocab_size

45035

In [47]:
# Calculate balanced class weights to handle class imbalance in a dataset.
class_weights_sent = compute_class_weight(class_weight='balanced', classes=np.unique(y['Sentiment']), y=y['Sentiment'])

# Convert weights to a dictionary format
class_weight_dict_sent = dict(enumerate(class_weights_sent))

# Print the class weight dictionary
print(class_weight_dict_sent)

{0: 2.0, 1: 1.0, 2: 0.6666666666666666}


In [48]:
# Calculate balanced class weights to handle class imbalance in a dataset.
class_weights_emo = compute_class_weight(class_weight='balanced', classes=np.unique(y['Emotion']), y=y['Emotion'])

# Convert weights to a dictionary format
class_weight_dict_emo = dict(enumerate(class_weights_emo))

# Print the class weight dictionary
print(class_weight_dict_emo)

{0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0}


In [49]:
# Convert y_train to a NumPy array if it's a pandas Series
y_train_sent = y_train['Sentiment'].values if isinstance(y_train['Sentiment'], pd.Series) else y_train['Sentiment']

# Convert y_val to a NumPy array if it's a pandas Series
y_val_sent = y_val['Sentiment'].values if isinstance(y_val['Sentiment'], pd.Series) else y_val['Sentiment']

# Convert y_test to a NumPy array if it's a pandas Series
y_test_sent = y_test['Sentiment'].values if isinstance(y_test['Sentiment'], pd.Series) else y_test['Sentiment']

In [50]:
# Convert y_train to a NumPy array if it's a pandas Series
y_train_emo = y_train['Emotion'].values if isinstance(y_train['Emotion'], pd.Series) else y_train['Emotion']

# Convert y_val to a NumPy array if it's a pandas Series
y_val_emo = y_val['Emotion'].values if isinstance(y_val['Emotion'], pd.Series) else y_val['Emotion']

# Convert y_test to a NumPy array if it's a pandas Series
y_test_emo = y_test['Emotion'].values if isinstance(y_test['Emotion'], pd.Series) else y_test['Emotion']

In [19]:
from tensorflow.keras import layers, models, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, Callback
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense,LayerNormalization
from tqdm import tqdm
import numpy as np
from tensorflow.keras.regularizers import l2
# Ensure Input Shape
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
# Define a TQDM Progress Bar Callback
class TQDMProgressBar(Callback):
    def on_train_begin(self, logs=None):
        self.epochs = self.params['epochs']
        self.epochs_bar = tqdm(total=self.epochs, desc='Training Progress', position=0)

    def on_epoch_end(self, epoch, logs=None):
        self.epochs_bar.update(1)
        self.epochs_bar.set_postfix(logs)

    def on_train_end(self, logs=None):
        self.epochs_bar.close()

# Common Hyperparameters
learning_rate = 1e-4
embedding_dim=64
# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
tqdm_callback = TQDMProgressBar()

In [20]:
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Input, Embedding, Conv1D, Dropout,AveragePooling1D, MaxPooling1D, LayerNormalization, LSTM, Bidirectional, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

# Define Sentiment Model
sentiment_inputs = Input(shape=(max_length,))
x = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(sentiment_inputs)

# Convolutional layer
x = Conv1D(filters=64, kernel_size=5, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.3)(x)

# Average Pooling layer
x = AveragePooling1D(pool_size=2)(x)
x = LayerNormalization()(x)

# Bidirectional LSTM layers
x = Bidirectional(LSTM(16, return_sequences=True, kernel_regularizer=l2(1e-5)))(x)
x = Bidirectional(LSTM(8, return_sequences=True, kernel_regularizer=l2(1e-5)))(x)
x = Dropout(0.5)(x)
# Flatten before Dense layers
x = layers.Flatten()(x)

# Dense Layers
x = Dense(16, activation='relu', kernel_regularizer=l2(1e-2))(x)

# Output Layer for Sentiment
sentiment_output = Dense(3, activation='softmax', name='sentiment_output')(x)

# Compile Sentiment Model
sentiment_model = Model(inputs=sentiment_inputs, outputs=sentiment_output)
sentiment_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=learning_rate),
    metrics=['accuracy']
)
sentiment_model.summary()

# Learning Rate Scheduler
reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss',        # Monitor validation loss
    factor=0.1,                # Reduce learning rate by this factor
    patience=5,                # Number of epochs with no improvement to wait
    min_lr=1e-6,               # Minimum learning rate
    verbose=1                  # Display reduction messages
)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 33)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 33, 64)         │     1,297,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 29, 64)         │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 29, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_1             │ (None, 14, 64)         │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ (None, 14, 64)         │           128 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 14, 32)         │        10,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 14, 16)         │         2,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 14, 16)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │         3,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sentiment_output (Dense)        │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,334,659 (5.09 MB)

 Trainable params: 1,334,659 (5.09 MB)

 Non-trainable params: 0 (0.00 B)

In [53]:
# Train Sentiment Model
sentiment_history = sentiment_model.fit(
    X_train,
    y_train_sent,  # Use Sentiment labels
    validation_data=(X_val, y_val_sent),
    epochs=200,
    batch_size=16,
    callbacks=[early_stopping, tqdm_callback,reduce_lr_on_plateau],
    class_weight=class_weight_dict_sent,
    verbose=1
)

Training Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1/200
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.4925 - loss: 1.1670

Training Progress:   0%|          | 1/200 [08:40<28:47:51, 520.96s/it, accuracy=0.557, loss=1.01, val_accuracy=0.672, val_loss=0.752]

5100/5100 ━━━━━━━━━━━━━━━━━━━━ 521s 100ms/step - accuracy: 0.4925 - loss: 1.1670 - val_accuracy: 0.6718 - val_loss: 0.7517 - learning_rate: 1.0000e-04
Epoch 2/200
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.7279 - loss: 0.6852

Training Progress:   1%|          | 2/200 [15:56<25:52:36, 470.49s/it, accuracy=0.752, loss=0.64, val_accuracy=0.769, val_loss=0.587]

...

Training Progress:   6%|▋         | 13/200 [1:39:10<23:06:49, 444.97s/it, accuracy=0.963, loss=0.133, val_accuracy=0.827, val_loss=0.651]

5100/5100 ━━━━━━━━━━━━━━━━━━━━ 445s 87ms/step - accuracy: 0.9629 - loss: 0.1330 - val_accuracy: 0.8269 - val_loss: 0.6509 - learning_rate: 1.0000e-05
Epoch 14/200
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.9644 - loss: 0.1269

Training Progress:   7%|▋         | 14/200 [1:46:24<22:49:05, 441.64s/it, accuracy=0.964, loss=0.129, val_accuracy=0.827, val_loss=0.645]


Epoch 14: ReduceLROnPlateau reducing learning rate to 1e-06.
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 434s 85ms/step - accuracy: 0.9644 - loss: 0.1269 - val_accuracy: 0.8271 - val_loss: 0.6446 - learning_rate: 1.0000e-05


Training Progress:   7%|▋         | 14/200 [1:46:24<23:33:46, 456.06s/it, accuracy=0.964, loss=0.129, val_accuracy=0.827, val_loss=0.645]


In [ ]:
# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
# Define Emotion Model
emotion_inputs = Input(shape=(max_length,))
x = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(emotion_inputs)
# Convolutional layer
x = Conv1D(filters=64, kernel_size=5, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.4)(x)

# Average Pooling layer
x = AveragePooling1D(pool_size=4)(x)
x = LayerNormalization()(x)

# Bidirectional LSTM layers
x = Bidirectional(LSTM(16, return_sequences=True, kernel_regularizer=l2(1e-5)))(x)
x = Bidirectional(LSTM(8, return_sequences=True, kernel_regularizer=l2(1e-5)))(x)
x = Dropout(0.5)(x)
# Flatten before Dense layers
x = layers.Flatten()(x)

# Dense Layers
x = Dense(32, activation='relu', kernel_regularizer=l2(1e-4))(x)

# Output Layer for Emotion
emotion_output = layers.Dense(6, activation='softmax', name='emotion_output')(x)

# Compile Emotion Model
emotion_model = models.Model(inputs=emotion_inputs, outputs=emotion_output)
emotion_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=learning_rate),
    metrics=['accuracy']
)


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
tqdm_callback = TQDMProgressBar()
# Train Emotion Model
emotion_history = emotion_model.fit(
    X_train,
    y_train_emo,  # Use Emotion labels
    validation_data=(X_val, y_val_emo),
    epochs=200,
    batch_size=16,
    callbacks=[early_stopping, tqdm_callback,reduce_lr_on_plateau],
    class_weight=class_weight_dict_emo,
    verbose=1
)


Training Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1/200
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.2517 - loss: 1.6543

Training Progress:   0%|          | 1/200 [05:37<18:38:57, 337.37s/it, accuracy=0.31, loss=1.53, val_accuracy=0.425, val_loss=1.35]

5100/5100 ━━━━━━━━━━━━━━━━━━━━ 337s 65ms/step - accuracy: 0.2518 - loss: 1.6543 - val_accuracy: 0.4252 - val_loss: 1.3492 - learning_rate: 1.0000e-04
Epoch 2/200
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.4893 - loss: 1.2350

Training Progress:   1%|          | 2/200 [10:33<17:13:37, 313.22s/it, accuracy=0.535, loss=1.16, val_accuracy=0.618, val_loss=1.03]

...

Training Progress:   8%|▊         | 15/200 [1:04:38<13:49:54, 269.16s/it, accuracy=0.927, loss=0.236, val_accuracy=0.749, val_loss=0.996]

5100/5100 ━━━━━━━━━━━━━━━━━━━━ 262s 51ms/step - accuracy: 0.9283 - loss: 0.2321 - val_accuracy: 0.7493 - val_loss: 0.9959 - learning_rate: 1.0000e-05
Epoch 16/200
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9290 - loss: 0.2302

Training Progress:   8%|▊         | 16/200 [1:09:02<13:40:06, 267.43s/it, accuracy=0.928, loss=0.23, val_accuracy=0.749, val_loss=1.02]  


Epoch 16: ReduceLROnPlateau reducing learning rate to 1e-06.
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 263s 52ms/step - accuracy: 0.9290 - loss: 0.2302 - val_accuracy: 0.7491 - val_loss: 1.0162 - learning_rate: 1.0000e-05


Training Progress:   8%|▊         | 16/200 [1:09:02<13:13:56, 258.89s/it, accuracy=0.928, loss=0.23, val_accuracy=0.749, val_loss=1.02]


In [56]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import numpy as np


In [57]:
# Predict class labels for sentiment
sentiment_predictions = sentiment_model.predict(X_test)
sentiment_predicted_labels = np.argmax(sentiment_predictions, axis=1)

319/319 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step


In [58]:
# Predict class labels for emotion
emotion_predictions = emotion_model.predict(X_test)
emotion_predicted_labels = np.argmax(emotion_predictions, axis=1)

319/319 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step


In [59]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

# True labels for sentiment
sentiment_true_labels = y_test_sent

# Ensure the sentiment map matches the number of classes
sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}

# Individual metrics
sentiment_accuracy = accuracy_score(sentiment_true_labels, sentiment_predicted_labels)
sentiment_f1 = f1_score(sentiment_true_labels, sentiment_predicted_labels, average='weighted')
sentiment_precision = precision_score(sentiment_true_labels, sentiment_predicted_labels, average='weighted')
sentiment_recall = recall_score(sentiment_true_labels, sentiment_predicted_labels, average='weighted')

# Display results
print(f"Accuracy: {sentiment_accuracy}")
print(f"F1 Score: {sentiment_f1}")
print(f"Precision: {sentiment_precision}")
print(f"Recall: {sentiment_recall}")


Accuracy: 0.7986274509803921
F1 Score: 0.8017075316411655
Precision: 0.8080247699310676
Recall: 0.7986274509803921


In [60]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

# True labels for emotion
emo_true_labels = y_test_emo

# Ensure the emotion map matches the number of classes
emotion_mapping = {
    'fear': 0,
    'Happiness': 1,
    'anger': 2,
    'joy' : 3,   
    'sadness': 4,    
    'neutral': 5, 
}

# Individual metrics
emo_accuracy = accuracy_score(emo_true_labels, emotion_predicted_labels)
emo_f1 = f1_score(emo_true_labels, emotion_predicted_labels, average='weighted')
emo_precision = precision_score(emo_true_labels, emotion_predicted_labels, average='weighted')
emo_recall = recall_score(emo_true_labels, emotion_predicted_labels, average='weighted')

# Display results
print(f"Accuracy: {emo_accuracy}")
print(f"F1 Score: {emo_f1}")
print(f"Precision: {emo_precision}")
print(f"Recall: {emo_recall}")

Accuracy: 0.7204901960784313
F1 Score: 0.7214743952699366
Precision: 0.7289946818299825
Recall: 0.7204901960784313


In [61]:
# True labels for sentiment
sentiment_true_labels = y_test_sent

# Ensure the sentiment map matches the number of classes
sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}

# Individual metrics
sentiment_accuracy = accuracy_score(sentiment_true_labels, sentiment_predicted_labels)
sentiment_f1 = f1_score(sentiment_true_labels, sentiment_predicted_labels, average='weighted')
sentiment_precision = precision_score(sentiment_true_labels, sentiment_predicted_labels, average='weighted')
sentiment_recall = recall_score(sentiment_true_labels, sentiment_predicted_labels, average='weighted')

# Display results
print(f"Accuracy: {sentiment_accuracy}")
print(f"F1 Score: {sentiment_f1}")
print(f"Precision: {sentiment_precision}")
print(f"Recall: {sentiment_recall}")


Accuracy: 0.7986274509803921
F1 Score: 0.8017075316411655
Precision: 0.8080247699310676
Recall: 0.7986274509803921


In [62]:
"""# Save the model to a .keras file
sentiment_model.save('sentiment_model_prototype.keras')
print("Model saved to 'sentiment_model_prototype.keras'")"""

'# Save the model to a .keras file\nsentiment_model.save(\'sentiment_model_prototype.keras\')\nprint("Model saved to \'sentiment_model_prototype.keras\'")'

In [63]:
"""# Save the model to a .keras file
emotion_model.save('emotion_model_prototype.keras')
print("Model saved to 'emotion_model_prototype.keras'")"""

'# Save the model to a .keras file\nemotion_model.save(\'emotion_model_prototype.keras\')\nprint("Model saved to \'emotion_model_prototype.keras\'")'

In [2]:
df=pd.read_csv("hate_speech_labeled_data.csv")  

In [3]:
df.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [4]:
import re
from nltk.tokenize import word_tokenize

def clean_text(text):
    # Check if text is a string (to avoid errors with NaNs)
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()
        
        # Remove URLs
        text = re.sub(r"http\S+|www\S+", '', text)
        
        # Remove hashtags and @usernames
        text = re.sub(r"(#[\d\w\.]+)", '', text)
        text = re.sub(r"(@[\d\w\.]+)", '', text)
        
        # Remove numbers (optional if numbers are relevant)
        text = re.sub(r"\d+", '', text)
        
        # **Remove underscores**
        text = text.replace('_', '')
        
        # Remove all special characters except valid punctuation (if needed)
        text = re.sub(r"[^\w\s]", '', text)
        
        # Remove non-ASCII characters
        text = re.sub(r"[^\x00-\x7F]+", '', text)
        
        # Remove extra whitespaces
        text = re.sub(r"\s+", ' ', text).strip()

        # Tokenization using nltk
        tokens = word_tokenize(text)

        # Remove stop words (if you have a defined stop words list)
        # Uncomment and define stop words if needed:
        # stop_words = set(stopwords.words('english'))
        # tokens = [word for word in tokens if word not in stop_words]

        # Return the cleaned and tokenized text
        return tokens
    
    return text  # Return the input as is if it's not a string


In [5]:
# Apply the cleaning function to the 'tweet' column
df['tweet'] = df['tweet'].apply(clean_text)

In [6]:
# Split 80% for training and 20% for temporary set (which will be split into validation and test sets)
X_train, X_temp, y_train, y_temp = train_test_split(df["tweet"], df['class'], test_size=0.2, random_state=42)

# Split the 20% temporary set into 10% validation and 10% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [7]:
# Define a mapping for sentiments to unique integer codes
hate_mapping = {
    'hate': 0,
    'offensive': 1,
    'neither': 2
}

In [8]:
# Initialize the tokenizer
tokenizer = Tokenizer()
# Fit the tokenizer on the texts
tokenizer.fit_on_texts(df["tweet"])
# Extract word index and vocabulary size
word_index = tokenizer.word_index

In [9]:
#Convert text into numerical values 

X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(X_test)
#calculate the max length of text
max_length = 0
for sequence in df['tweet']:
    sequence_length = len(sequence)
    if sequence_length > max_length:
        max_length = sequence_length


print(max_length)

33


In [10]:
#Adding zeros at the beginning
X_train = pad_sequences(X_train,maxlen=max_length)
X_val = pad_sequences(X_val,maxlen=max_length)
X_test = pad_sequences(X_test,maxlen=max_length)

In [12]:
from keras.callbacks import ReduceLROnPlateau
#Calculate the vocabulary size
vocab_size = len(word_index) + 1
class TQDMProgressBar(Callback):
    def on_train_begin(self, logs=None):
        self.epochs = self.params['epochs']
        self.epochs_bar = tqdm(total=self.epochs, desc='Training Progress', position=0)

    def on_epoch_end(self, epoch, logs=None):
        self.epochs_bar.update(1)
        self.epochs_bar.set_postfix(logs)

    def on_train_end(self, logs=None):
        self.epochs_bar.close()
# Learning Rate Scheduler
reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss',        # Monitor validation loss
    factor=0.1,                # Reduce learning rate by this factor
    patience=5,                # Number of epochs with no improvement to wait
    min_lr=1e-6,               # Minimum learning rate
    verbose=1                  # Display reduction messages
)
# Common Hyperparameters
learning_rate = 1e-4
embedding_dim = 32
# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
tqdm_callback = TQDMProgressBar()

In [13]:
# Calculate balanced class weights to handle class imbalance in a dataset.
class_weights_hate_speech = compute_class_weight(class_weight='balanced', classes=np.unique(df['class']), y=df['class'])

# Convert weights to a dictionary format
class_weight_dict_hate_speech = dict(enumerate(class_weights_hate_speech))

# Print the class weight dictionary
print(class_weight_dict_hate_speech)

{0: 5.776923076923077, 1: 0.43048462741010945, 2: 1.9843862599087196}


In [14]:
# Convert y_train to a NumPy array if it's a pandas Series
y_train = y_train.values if isinstance(y_train, pd.Series) else y_train

# Convert y_val to a NumPy array if it's a pandas Series
y_val = y_val.values if isinstance(y_val, pd.Series) else y_val

# Convert y_test to a NumPy array if it's a pandas Series
y_test = y_test.values if isinstance(y_test, pd.Series) else y_test

In [15]:
"""# Ensure Input Shape
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)"""

'# Ensure Input Shape\nX_train = np.expand_dims(X_train, axis=-1)\nX_val = np.expand_dims(X_val, axis=-1)\nX_test = np.expand_dims(X_test, axis=-1)'

In [16]:
print(np.array(df["tweet"].shape))
print(np.array(df['class'].shape))

[24783]
[24783]


In [17]:
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import layers, models, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, Callback
from keras.models import Model
from keras.regularizers import l2
from keras.layers import Input, Embedding, Conv1D, Dropout,AveragePooling1D, MaxPooling1D, LayerNormalization, LSTM, Bidirectional, Dense
# Define hate/offensive Model
hate_speech_inputs = Input(shape=(max_length,))
x = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(hate_speech_inputs)
# Convolutional layer
x = Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.3)(x)

# Max Pooling layer
x = MaxPooling1D(pool_size=2)(x)
x = LayerNormalization()(x)

# LSTM layers
x = Bidirectional(LSTM(16, return_sequences=True, kernel_regularizer=l2(1e-5)))(x)
x = Bidirectional(LSTM(8, return_sequences=True, kernel_regularizer=l2(1e-5)))(x)
x = Dropout(0.5)(x)
# Flatten before Dense layers
x = layers.Flatten()(x)

# Dense Layers
x = Dense(16, activation='relu', kernel_regularizer=l2(1e-4))(x)

# Output Layer for hate/offensive
hate_speech_output = layers.Dense(3, activation='softmax', name='hate_speech_output')(x)

# Compile hate/offensive Model
hate_speech_model = models.Model(inputs=hate_speech_inputs, outputs=hate_speech_output)
hate_speech_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=learning_rate),
    metrics=['accuracy']
)
hate_speech_model.summary()
# Train hate/offensive Model
hate_speech_history = hate_speech_model.fit(
    X_train,
    y_train,  
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=16,
    callbacks=[early_stopping, tqdm_callback,reduce_lr_on_plateau],
    class_weight=class_weight_dict_hate_speech,
    verbose=1
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 33)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 33, 32)         │       648,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 31, 32)         │         3,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 31, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 15, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 15, 32)         │            64 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 15, 32)         │         6,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 15, 16)         │         2,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 15, 16)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 240)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │         3,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hate_speech_output (Dense)      │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 664,643 (2.54 MB)

 Trainable params: 664,643 (2.54 MB)

 Non-trainable params: 0 (0.00 B)

...

Training Progress:   7%|▋         | 14/200 [06:16<1:28:39, 28.60s/it, accuracy=0.915, loss=0.182, val_accuracy=0.788, val_loss=0.695]

1240/1240 ━━━━━━━━━━━━━━━━━━━━ 29s 23ms/step - accuracy: 0.9129 - loss: 0.1799 - val_accuracy: 0.7881 - val_loss: 0.6952 - learning_rate: 1.0000e-05
Epoch 15/200
1238/1240 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9199 - loss: 0.1739

Training Progress:   8%|▊         | 15/200 [06:45<1:28:27, 28.69s/it, accuracy=0.919, loss=0.178, val_accuracy=0.776, val_loss=0.711]


Epoch 15: ReduceLROnPlateau reducing learning rate to 1e-06.
1240/1240 ━━━━━━━━━━━━━━━━━━━━ 29s 23ms/step - accuracy: 0.9199 - loss: 0.1739 - val_accuracy: 0.7764 - val_loss: 0.7112 - learning_rate: 1.0000e-05


Training Progress:   8%|▊         | 15/200 [06:45<1:23:21, 27.04s/it, accuracy=0.919, loss=0.178, val_accuracy=0.776, val_loss=0.711]


In [18]:
# Save the model to a .keras file
hate_speech_model.save('hate_speech_model_prototype.keras')
print("Model saved to 'hate_speech_model_prototype.keras'")

Model saved to 'hate_speech_model_prototype.keras'


In [19]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
# Predict class labels for hate/offensive
hate_predictions = hate_speech_model.predict(X_test)
hate_predicted_labels = np.argmax(hate_predictions, axis=1)
# True labels for hate/offensive
hate_true_labels =y_test
# Define a mapping for hate/offensive to unique integer codes
hate_mapping = {
    'hate': 0,
    'offensive': 1,
    'neither': 2
}

# Individual metrics
hate_accuracy = accuracy_score(hate_predicted_labels,hate_true_labels)
hate_f1 = f1_score(hate_true_labels, hate_predicted_labels, average='weighted')
hate_precision = precision_score(hate_true_labels, hate_predicted_labels, average='weighted')
hate_recall = recall_score(hate_true_labels, hate_predicted_labels, average='weighted')

# Display results
print(f"Accuracy: {hate_accuracy}")
print(f"F1 Score: {hate_f1}")
print(f"Precision: {hate_precision}")
print(f"Recall: {hate_recall}")

78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step
Accuracy: 0.7676482452601856
F1 Score: 0.7947524059682408
Precision: 0.8381895166367666
Recall: 0.7676482452601856
